In [78]:
#!pip3 install selenium
#!pip3 install unidecode
# before test this code, make download geckodriver and copy this file for /usr/local/bin
# sudo cp geckodriver /usr/local/bin

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
import time # it was necessary to trigger the click time on the buttons
import unidecode

In [2]:
# Globo esporte url's
athletes_url = "https://interativos.globoesporte.globo.com/estatisticas/atletas/"
teams_url = "https://interativos.globoesporte.globo.com/estatisticas/times/"

In [3]:
path_teams_files = '../Outputs/teams'
path_athletes_files = '../Outputs/athletes'

In [4]:
labels = ['Gol marcado','Gols sofridos','Gols contra','Gols de falta',
              'Gol de pênalti','Gols dentro da área','Gol de fora da área','Finalização na trave',
              'Finalizações para fora','Faltas recebidas', 'Faltas cometidas',
             'Impedimentos','Cartões amarelos','Cartões vermelhos', 'Defesas difíceis',
             'Defesas de pênaltis', 'Roubadas de bola','Passes errados']


In [5]:
def create_normalize_labels_dic(labels_list):
    dic = {}
    for item in labels_list:
        label = item.replace(" ", "_")
        label = label.lower()
        label = unidecode.unidecode(label) 
        dic[item] = label   
    return dic

In [6]:
titles = create_normalize_labels_dic(labels)

In [8]:
def extract_data(url):
    
    driver.get(url)
    driver.implicitly_wait(10)
    
    # click in vermais button to open element in html body
    for title in titles: 
        driver.find_element_by_xpath(f"//button[@data-title='{title}']").click()
        time.sleep(4)
    
    time.sleep(10) # this time is necessary for opening some elements on the screen
    
    # get section ranking
    div_criterio = driver.find_element_by_xpath("//div[@class='criterio']")
    html_content = div_criterio.get_attribute('outerHTML')
    soup = BeautifulSoup(html_content, 'html.parser')
    section_class_ranking_list = soup.find_all(class_='ranking')
    return section_class_ranking_list

In [9]:
def build_teams_csv_files(section_list, path):
    
    for item in section_list:
        scoutname = item['data-scoutnome']
        bucket = []
        figure_list = item.find_all('figure')
    
        for figure in figure_list[1:]:

            name = figure.find(class_='ranking__nome').text
            name = name.strip()

            div_number_of_games = figure.find(class_='ranking__results--item ranking__results--jogos')
            spans = div_number_of_games.find_all('span')
            number_of_games = spans[1].text

            value = figure.find(class_='ranking__results--item ranking__results--total').text

            temp_list = []
            temp_list.extend([name, number_of_games, value ])
            bucket.append(temp_list)
        
        name_file = titles[scoutname]
        df = pd.DataFrame(bucket, columns=['name', 'number_of_games', name_file])
        df.to_csv(f'{path}/{name_file}.csv', index=False)

In [36]:
def build_athletes_csv_files(section_list, path):
    
    for item in section_list:
        scoutname = item['data-scoutnome']
        bucket = []
        figure_list = item.find_all('figure')
    
        for figure in figure_list[1:]:

            team = figure.find(class_='ranking__escudo--image')
            team = team['title'].strip()

            position = figure.find(class_='ranking__posicao').text
            position = position.strip()

            name = figure.find(class_='ranking__nome').text
            name = name.strip()

            div_number_of_games = figure.find(class_='ranking__results--item ranking__results--jogos')
            spans = div_number_of_games.find_all('span')
            number_of_games = spans[1].text

            value = figure.find(class_='ranking__results--item ranking__results--total').text

            temp_list = []
            temp_list.extend([name, position, team , number_of_games, value ])
            bucket.append(temp_list)
        
        name_file = titles[scoutname]
        df = pd.DataFrame(bucket, columns=['name', 'position', 'team', 'number_of_games',name_file])
        df.to_csv(f'{path}/{name_file}.csv', index=False)

In [11]:
option = Options()
option.headless = True
driver = webdriver.Firefox(options=option)
teams_section_list = extract_data(url=teams_url)
athletes_section_list = extract_data(url=athletes_url)
driver.quit()

In [32]:

build_teams_csv_files(teams_section_list, path=path_teams_files)

In [37]:
build_athletes_csv_files(athletes_section_list, path=path_athletes_files)